# Google Play Apps and Reviews

In [1]:
import pandas as pd
apps_with_duplicates = pd.read_csv('processed-android/apps.csv')

# Borrar duplicados de apps_with_duplicates
apps = apps_with_duplicates.drop_duplicates()

# Mostrar el número total de apps
print('Número total de apps en el dataset = ',apps.count())

# Mirar los primeros cinco registros
print(apps.head())

Número total de apps en el dataset =  Unnamed: 0        9659
App               9659
Category          9659
Rating            8196
Reviews           9659
Size              8432
Installs          9659
Type              9659
Price             9659
Content Rating    9659
Genres            9659
Last Updated      9659
Current Ver       9651
Android Ver       9657
dtype: int64
   Unnamed: 0                                                App  \
0           0     Photo Editor & Candy Camera & Grid & ScrapBook   
1           1                                Coloring book moana   
2           2  U Launcher Lite – FREE Live Cool Themes, Hide ...   
3           3                              Sketch - Draw & Paint   
4           4              Pixel Draw - Number Art Coloring Book   

         Category  Rating  Reviews  Size     Installs  Type Price  \
0  ART_AND_DESIGN     4.1      159  19.0      10,000+  Free     0   
1  ART_AND_DESIGN     3.9      967  14.0     500,000+  Free     0   
2  ART_AND_